In [ ]:
# Cell 1: Imports
import json
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import interpolate
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tqdm import tqdm

print("✅ Imports loaded successfully!")

In [ ]:
# Cell 2: Constants and Helper Functions
TARGET_FRAMES = 30
N_FACE = 468
N_POSE = 33
N_LEFT_HAND = 21
N_RIGHT_HAND = 21
TOTAL_LANDMARKS = N_FACE + N_POSE + N_LEFT_HAND + N_RIGHT_HAND

def load_parquet_landmarks(file_path):
    try:
        return pd.read_parquet(file_path)
    except:
        return None

def parquet_to_array(df):
    if df is None or len(df) == 0:
        return None
    n_frames = df['frame'].max() + 1
    data = np.zeros((n_frames, TOTAL_LANDMARKS, 3), dtype=np.float32)
    for _, row in df.iterrows():
        frame_idx = int(row['frame'])
        lm_idx = int(row['landmark_index'])
        lm_type = row['type']
        if lm_type == 'face':
            global_idx = lm_idx
        elif lm_type == 'left_hand':
            global_idx = N_FACE + lm_idx
        elif lm_type == 'pose':
            global_idx = N_FACE + N_LEFT_HAND + lm_idx
        elif lm_type == 'right_hand':
            global_idx = N_FACE + N_LEFT_HAND + N_POSE + lm_idx
        else:
            continue
        if global_idx < TOTAL_LANDMARKS and frame_idx < n_frames:
            data[frame_idx, global_idx, 0] = row['x']
            data[frame_idx, global_idx, 1] = row['y']
            data[frame_idx, global_idx, 2] = row['z']
    return data

def resample_sequence(data, target_len=TARGET_FRAMES):
    current_len = data.shape[0]
    if current_len == target_len:
        return data
    old_indices = np.linspace(0, current_len - 1, current_len)
    new_indices = np.linspace(0, current_len - 1, target_len)
    resampled = np.zeros((target_len, TOTAL_LANDMARKS, 3), dtype=np.float32)
    for lm in range(TOTAL_LANDMARKS):
        for coord in range(3):
            f = interpolate.interp1d(old_indices, data[:, lm, coord], kind='linear')
            resampled[:, lm, coord] = f(new_indices)
    return resampled

print(f"✅ Helper functions defined (Target frames: {TARGET_FRAMES}, Landmarks: {TOTAL_LANDMARKS})")

In [ ]:
# Cell 3: Load Label Mapping
LABEL_MAPPING_PATH = "/kaggle/input/wlasl200-mapping/label_mapping.json"

with open(LABEL_MAPPING_PATH, 'r') as f:
    label_data = json.load(f)

gloss_to_label = label_data['gloss_to_label']
label_to_gloss = label_data['label_to_gloss']
num_classes = label_data['num_classes']

print(f"✅ Loaded {num_classes} word labels")
print(f"   First 10: {list(gloss_to_label.keys())[:10]}")
print(f"   Last 10: {list(gloss_to_label.keys())[-10:]}")

In [ ]:
# Cell 4: Create Video-to-Word Mapping
WLASL_DIR = "/kaggle/input/wlasl2000-landmarks/wlasl_landmarks"
WLASL_FULL = "/kaggle/input/wlasl-processed/WLASL_v0.3.json"

with open(WLASL_FULL, 'r') as f:
    wlasl_full = json.load(f)

video_to_word = {}
word_to_videos = {}

for entry in tqdm(wlasl_full, desc="Creating mapping"):
    word = entry.get('gloss', None)
    if word and word in gloss_to_label:
        instances = entry.get('instances', [])
        for instance in instances:
            video_id = instance.get('video_id', None)
            if video_id:
                video_to_word[video_id] = word
                if word not in word_to_videos:
                    word_to_videos[word] = []
                word_to_videos[word].append(video_id)

print(f"✅ Mapping created: {len(video_to_word)} videos for {len(word_to_videos)} words")
video_counts = [len(videos) for videos in word_to_videos.values()]
print(f"   Videos per word: min={min(video_counts)}, max={max(video_counts)}, mean={np.mean(video_counts):.1f}")

In [ ]:
# Cell 5: Load Dataset
X_data = []
y_data = []

for word, videos in tqdm(word_to_videos.items(), desc="Loading dataset"):
    label = gloss_to_label[word]
    for video_id in videos:
        file_path = f"{WLASL_DIR}/{video_id}.parquet"
        if not os.path.exists(file_path):
            continue
        df = load_parquet_landmarks(file_path)
        if df is None:
            continue
        landmarks = parquet_to_array(df)
        if landmarks is None:
            continue
        landmarks_resampled = resample_sequence(landmarks, TARGET_FRAMES)
        landmarks_flat = landmarks_resampled.reshape(TARGET_FRAMES, -1)
        X_data.append(landmarks_flat)
        y_data.append(label)

X_data = np.array(X_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.int32)

print(f"✅ Dataset loaded: {len(X_data)} samples")
print(f"   X shape: {X_data.shape}")
print(f"   Unique classes: {len(np.unique(y_data))}")

In [ ]:
# Cell 6: Train/Val Split
X_train, X_val, y_train, y_val = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)

print("✅ Train/Val split:")
print(f"   Train: {len(X_train)} samples")
print(f"   Val: {len(X_val)} samples")

In [ ]:
# Cell 7: Build LSTM Model
INPUT_SHAPE = (TARGET_FRAMES, TOTAL_LANDMARKS * 3)

model = Sequential([
    LSTM(64, return_sequences=True, activation='relu', input_shape=INPUT_SHAPE),
    Dropout(0.2),
    LSTM(128, return_sequences=True, activation='relu'),
    Dropout(0.2),
    LSTM(64, return_sequences=False, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ LSTM Model built")
print(f"   Input: {INPUT_SHAPE}")
print(f"   Output: {num_classes} classes")
print(f"   Parameters: {model.count_params():,}")
model.summary()

In [ ]:
# Cell 8: Train Model
checkpoint = ModelCheckpoint(
    '/kaggle/working/lstm_wlasl100_best.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint, reduce_lr, early_stop],
    verbose=1
)

print("✅ Training complete!")

In [ ]:
# Cell 9: Evaluate Model
best_model = tf.keras.models.load_model('/kaggle/working/lstm_wlasl100_best.h5')

val_loss, val_acc = best_model.evaluate(X_val, y_val, verbose=0)
y_pred = best_model.predict(X_val, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
top5_acc = top_k_accuracy_score(y_val, y_pred, k=5)

print("✅ Final Results:")
print(f"   Val Loss: {val_loss:.4f}")
print(f"   Val Accuracy: {val_acc*100:.2f}%")
print(f"   Top-5 Accuracy: {top5_acc*100:.2f}%")

print("\n🔍 Sample predictions:")
for i in range(min(10, len(y_val))):
    true_word = label_to_gloss[str(y_val[i])]
    pred_word = label_to_gloss[str(y_pred_classes[i])]
    confidence = y_pred[i][y_pred_classes[i]] * 100
    status = "✅" if y_val[i] == y_pred_classes[i] else "❌"
    print(f"   {status} True: {true_word:15s} | Pred: {pred_word:15s} ({confidence:.1f}%)")

In [ ]:
# Cell 10: Save Model and Labels
model.save('/kaggle/working/lstm_wlasl100_final.h5')
model.save('/kaggle/working/lstm_wlasl100_savedmodel')

output_mapping = {
    'gloss_to_label': gloss_to_label,
    'label_to_gloss': label_to_gloss,
    'num_classes': num_classes,
    'model_info': {
        'input_shape': list(INPUT_SHAPE),
        'target_frames': TARGET_FRAMES,
        'total_landmarks': TOTAL_LANDMARKS,
        'val_accuracy': float(val_acc),
        'val_loss': float(val_loss)
    }
}

with open('/kaggle/working/lstm_wlasl100_labels.json', 'w') as f:
    json.dump(output_mapping, f, indent=2)

print("✅ Saved:")
print("   - lstm_wlasl100_final.h5")
print("   - lstm_wlasl100_savedmodel/")
print("   - lstm_wlasl100_labels.json")
print("\n📥 Download these files for local webcam inference")